## Урок 3. Системы управления базами данных MongoDB и SQLite в Python

#### Домашнее задание 

**1.** Развернуть  у  себя  на  компьютере/виртуальной  машине/хостинге  MongoDB  и  реализовать 
функцию, записывающую собранные вакансии в созданную БД. 

**2.** Написать  функцию,  которая  производит  поиск  и  выводит  на  экран  вакансии  с  заработной 
платой больше введённой суммы. 

**3.** Написать  функцию, которая будет добавлять в вашу базу данных только новые вакансии с 
сайта. 


### Пояснение к решению.

Т.к. в третьем задании возникает необходимость частично дублируется операция по поиску вакансии, то чтобы 
не увеличивать сократить  повоторять операции 

Последовательность выполнения операций переосмысленна. 
Логика работы программы теперь следующая:
1. Сначасла создается (если еще не была создана) БД `vacancy_db`. 
2. Получает из БД id всех вакансий и складывает из в массив `vacancy_id`. 
3. Далее, вызывается поиск вакансий на сайте **HH.ru**.  Поиск вакансии осуществляется внутри функции `vacancyes_on_hh`. В качестве парамеров в нее прередаются строка с названием вакансии `vacancy_name` и массив `vacancy_id`. Функция проходится по всем найденным вакансиям. И если id найденных вакансий еще нет в `vacancy_id`, то добавляет их в словарь `vacancy`.  
4. Словарь `vacancy`, полученный в результате работы функции `vacancyes_on_hh` добаваляется к существующим записям в БД. 

5. И, в качестве вишенки на торте, выполняем в БД выборку вакансий с зарплатой от 10000 руб. 

-----------------------------------------------------
В блокноте реализовал поиск только для HH.ru. 

Относительно кода из предыдущего урока.
  *  Добавилена проверка на None для всех полей попадающих в словарь.
  *  Поиск определение суммы зарплаты теперь выполняеся с помощью регулярного выражения.
  *  Сама процесс поиска теперь вынесен в отдельную процедуру. 

In [19]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import json
import re

In [20]:
from pymongo import MongoClient

In [21]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
    'charset': 'utf-8'
}

main_link = 'https://izhevsk.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true'

In [22]:
text = 'C++ Developer'  # 'Machine+Learning+Engineer'

In [23]:
def selary_analizer(selary_txt):  # возвращает массив содержащий значение зарбплаты.
    min_selary = None
    max_selary = None
    money_type = None
    symbols = ['от', 'до']

    if selary_txt != '' and selary_txt != 'По договорённости':  # 'проверяем наличие символов в строке'
        selary_txt = selary_txt.replace('—\xa0', '')
        selary_txt = selary_txt.replace('-', ' ')
        selary_txt = selary_txt.replace('\xa0', ' ')
        selary_txt = selary_txt.replace('./месяц', '')
        selary_array = selary_txt.split(' ')
        money_type = selary_array[-1].rstrip('.')

        if selary_array[0] == symbols[0]:  # от
            min_selary = re.findall(r'\d{1,8}', selary_array[1] + selary_array[2])[0]
        elif selary_array[0] == symbols[1]:  # до
            max_selary = re.findall(r'\d{1,8}', selary_array[1] + selary_array[2])[0]
        elif (len(selary_array) < 4):  # до
            max_selary = re.findall(r'\d{1,8}', selary_array[0] + selary_array[1])[0]
        else:  # -
            min_selary = re.findall(r'\d{1,8}',selary_array[0] + selary_array[1])[0]
            max_selary = re.findall(r'\d{1,8}',selary_array[2] + selary_array[3])[0]

    return [min_selary, max_selary, money_type]

#### Поиск Вакансии на сайте  https://hh.ru

In [24]:
def vacancyes_on_hh(follow_vacancy,id_list): 
    
    vacancy = [] # Словарь в котором будет храниться список вакансий.  
    page = 0
    is_next_page = True  # Флаг поназываюций что есть еще страницы.
    errors = 0
    new_vacancy = 0
    i = 0
    while is_next_page == True:
        response = requests.get(f'{main_link}&text={follow_vacancy}&page={page}', headers=header)
#       print(' ')
#       print(response.url)
        soup = bs(response.text, 'lxml')

        btn_next = soup.find('a', {'class': "HH-Pager-Controls-Next"})

        if btn_next != None:
            page += 1
        else:
            is_next_page = False

        vacancy_block = soup.find('div', {'class': 'vacancy-serp'})
        divs = vacancy_block.findChildren(recursive=False)


        for vk in divs:
            vacancy_data = {}
            try:
                if vk.attrs['class'][0] == 'vacancy-serp-item':
                    i += 1
                    # Ссылка на вакансию.
                    vacancy_id = None
                    vk_href = vk.find('a', {'class': 'bloko-link HH-LinkModifier'})

                    if vk_href is not None:
                        vk_href = vk_href['href']
                        vacancy_id = re.findall(r'\d{1,8}', vk_href)[0]
                        
                        if vacancy_id in id_list: # Прверяем сущесвует ли уже запись о даной вакансии в БД. 
                            continue
                        
                    vacancy_name = vk.find('div', {'class': 'vacancy-serp-item__info'})
                    if vacancy_name is not None:
                        vacancy_name = vacancy_name.getText()

                    min_selary, max_selary, money_type = None,None,None
                    selary_txt = vk.find('div', {'class': 'vacancy-serp-item__sidebar'})
                    if selary_txt is not None:
                        selary_txt = selary_txt.getText()
                        min_selary, max_selary, money_type = selary_analizer(selary_txt)

                    # Название фирмы.
                    vk_employer = vk.find('a', {'data-qa': "vacancy-serp__vacancy-employer"})
                    if vk_employer is not None:
                        vk_employer = vk_employer.getText()

                    vk_employer_id =vk.find('a', {'data-qa': "vacancy-serp__vacancy-employer"})
                    if vk_employer_id is not None:
                        vk_employer_id ='https://izhevsk.hh.ru' + vk_employer_id['href']

                    # Город расположения.
                    vk_location = vk.find('span', {'data-qa': "vacancy-serp__vacancy-address"})
                    if vk_location is not None:
                        vk_location = vk_location.getText()

                    # Дата публикации.
                    vacancy_date = vk.find('span', {'data-qa': "vacancy-serp__vacancy-date"})
                    if vacancy_date is not None:
                        vacancy_date = vacancy_date.getText()
                    
                    if min_selary is not None:
                        min_selary = float(min_selary)
                    
                    if max_selary is not None:
                        max_selary = float(max_selary)
                       
                    new_vacancy += 1
                    
                    vacancy_data['name'] = vacancy_name
                    vacancy_data['employer'] = vk_employer
                    vacancy_data['location'] = vk_location
                    vacancy_data['min_selary'] = min_selary
                    vacancy_data['max_selary'] = max_selary
                    vacancy_data['money_type'] = money_type
                    # vacancy_data['publicatin_date'] = vacancy_date
                    vacancy_data['href'] = vk_href
                    vacancy_data['employer_id'] = vk_employer_id
                    vacancy_data['source'] = 'https://hh.ru/'
                    vacancy_data['vacancy_id'] = vacancy_id
                    vacancy.append(vacancy_data)

            except Exception as ex:
                 errors += 1   

            print("", end="\r")
            print(f'Найдено вакансий: {i}',end=" ")
    print(' ')
    print(f'Новых вакансий : {new_vacancy}')
    print(f'Страниц :{page + 1}')
    print(f'Ошибок: {errors}' )
   
    
    return vacancy

1. Создается (если еще не была создана) БД vacancy_db.

In [25]:
client = MongoClient('localhost',27017)

In [26]:
db = client['vacancy_db']

In [27]:
hh_collection = db.vacancy_db

2. Получаем список всех id для конкретной вакансии. 

In [28]:
vc1 = hh_collection.find({},{'_id':0,'vacancy_id':1})

id_list = [i['vacancy_id'] for i in vc1] 

In [29]:
print(f'Число вакансий для hh хранящихся в БД: {len(id_list)}')

Число вакансий для hh хранящихся в БД: 2000


3. Выполняем поиск вакансий, соответствующих заданным критериям

In [30]:
vacancy = vacancyes_on_hh(text,id_list)

Найдено вакансий: 2000  
Новых вакансий : 158
Страниц :40
Ошибок: 0


In [31]:
len(vacancy)

158

4. Записываем новые найденные новые вакании в БД 

In [32]:
hh_collection.insert_many(vacancy)

5. Выполняем поиск вакансий с зарбплатой выше заданной суммы. Предварительно выполнив перево зарплаты из рублей в доллары и евро. 

In [33]:
follow_selary = 1000

# Выполняем перевод
selary_rub = follow_selary
# Рублей в доллоры 
selary_us = 1.0/71.28 * follow_selary
# Рублей в евро
selary_eur = 1.0/80.00 * follow_selary

print(f'selary_rub: {selary_rub:.02f}')
print(f'selary_USD: {selary_us:.02f}')
print(f'selary_EUR: {selary_eur:.02f}')

selary_rub: 1000.00
selary_USD: 14.03
selary_EUR: 12.50


In [34]:
vc = hh_collection.find({'$or':[
                         {'$and':[{'max_selary':{'$gt':selary_rub}},{'money_type':'руб'}]},
                         {'$and':[{'max_selary':{'$gt':selary_eur}},{'money_type':'EUR'}]},
                         {'$and':[{'max_selary':{'$gt':selary_us}},{'money_type':'USD'}]}]},
                         {'_id':0,'href':0,'employer_id':0}
                        ).limit(5)
pd.DataFrame(vc)

,name,employer,location,min_selary,max_selary,money_type,source,vacancy_id
0,Программист С++,Graebert GmbH,"Санкт-Петербург, Горьковская",2300.0,2800.0,USD,https://hh.ru/,189310
1,Коммерческий директор,SIA Egamings,Москва,3000.0,7000.0,EUR,https://hh.ru/,37741055
2,Руководитель отдела продаж / Директор по продажам,Zoletto,Москва,150000.0,270000.0,руб,https://hh.ru/,37713537
3,Field Marketing,"EPLAN, Компания","Москва, Аэропорт",100000.0,150000.0,руб,https://hh.ru/,37764604
4,Программист С++/Драйверист,Searchinform,Ростов-на-Дону,100000.0,150000.0,руб,https://hh.ru/,36712554


In [35]:
# hh_collection.drop()